In [2]:
from school import SchoolData
import pandas as pd
from typing import Dict

In [3]:
sd = SchoolData('1020080035')

In [10]:
data = sd.internet()

In [21]:
new_shape_dict = dict()
for group_x, df_x in pd.DataFrame(data).groupby([0]):
  new_shape_dict[group_x]: Dict = dict()
  group_x_dict = new_shape_dict[group_x]
  for group_y, df_y in df_x.groupby(1):
    values = df_y[2].values.tolist()
    if len(values) == 1:
      values = values[0]
    group_x_dict[group_y] = values

new_shape_dict

{'ระบบเครือข่าย': {'ระบบเครือข่าย (LAN):': 'มีระบบเครือข่าย LAN',
  'ระบบเครือข่ายไร้สาย (Wireless LAN):': 'มีระบบเครือข่ายไร้สาย (Wireless LAN)'},
 'ระบบเครือข่ายอินเทอร์เน็ตที่โรงเรียนเช่าเอง': {'ความเร็ว:': '-',
  'งบประมาณ/เดือน:': '-',
  'ประเภท:': '-',
  'ผู้ให้บริการ:': '-',
  'สถานะการใช้งาน:': '-'}}

In [40]:
computer_data: Dict = dict()
for group_x, df_x in pd.DataFrame(computer).groupby([0]):
  computer_data[group_x]: Dict = dict()
  group_x_dict = computer_data[group_x]
  for group_y, df_y in df_x.groupby(1):
    values = df_y[2].values.tolist()
    if len(values) == 1:
      values = values[0]
    group_x_dict[group_y] = values

In [41]:
computer_data

{'จำนวนคอมพิวเตอรทั้งหมด': {'คอมพิวเตอร์:': '78 เครื่อง',
  'ใช้งานได้:': '45 เครื่อง',
  'ใช้งานไม่ได้:': '33 เครื่อง'},
 'จำนวนคอมพิวเตอร์เพื่อการบริหารจัดการ': {'งบประมาณ สพฐ.': '0 เครื่อง',
  'จัดหาเอง/บริจาค:': '4 เครื่อง',
  'รวม': '4 เครื่อง',
  'ใช้งานได้:': '4 เครื่อง',
  'ใช้งานไม่ได้:': '0 เครื่อง'},
 'จำนวนคอมพิวเตอร์เพื่อการเรียนการสอน': {'งบประมาณ สพฐ.': '47 เครื่อง',
  'จัดหาเอง/บริจาค:': '27 เครื่อง',
  'รวม': '74 เครื่อง',
  'ใช้งานได้:': '41 เครื่อง',
  'ใช้งานไม่ได้:': '33 เครื่อง'}}